In [240]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd

In [228]:
def url_request(url):
    #this function takes the url and returns a Beautiful Soup object
    response = requests.get(url)
    if not response.ok:
        print('Server responded:', response.status_code)
    else:
        soup = BeautifulSoup(response.text, 'lxml')

    return soup


    

In [229]:
def get_detail_data(soup):
    #this function gets the datapoints from the BeatifulSoup object and returns a dictionary of the datapoints
    try:
        title = soup.find('h1', id = 'itemTitle').get_text().replace('\xa0','')
    except:
        title = ''
    
    try:
        price = soup.find('span', id = 'prcIsum').get('content')
    except:
        price = ''
    
    try:
        #units = soup.find('div', class_ = 'w2b-cnt w2b-3 w2b-brdr').text.strip().split(' ')[0]
        units = soup.find('a', class_ = 'vi-txt-underline').get_text().split(' ')[0]
    except:
        units = ''
    
    try:
        ratings_num = soup.find('a', id = '_rvwlnk').get_text().strip()
    except:
        ratings_num = ''
    
    try:
        seller_name = soup.find('span', class_ = 'mbg-nw').get_text().strip()
    except:
        seller_name = ''
    
    try:
        seller_ratings = soup.find('span', class_ = 'mbg-l').get_text().strip().replace('\n)','').replace('(','')
    except:
        seller_ratings = ''
    
    try:
        seller_feedback_score = soup.find('div', id = 'si-fb').get_text().strip().split('%')[0]
    except:
        seller_feedback_score = ''
        
    try:
        condition1 = soup.find('span', id = 'vi-cond-addl-info').get_text()
    except:
        condition1 = ''
    
    try:
        brand = soup.find('h2', itemprop = 'brand').get_text()
    except:
        brand = ''
        
    try:
        color = soup.find('h2', itemprop = 'color').get_text()
    except:
        color = ''
    
    try:
        viewed_ph = soup.find('div', id = 'vi_notification_new').get_text().replace('\n','')
    except:
        viewed_ph = ''
    
    
    
        
    data = {'name': title,
            'price': price,
            'units': units,
            'ratings_num': ratings_num,
            'seller_name': seller_name,
            'seller_ratings': seller_ratings,
            'seller_feedback_score': seller_feedback_score,
            'condition1': condition1,
            'brand': brand,
            'color': color,
            'viewed_ph': viewed_ph}
        
    return data

In [230]:
def get_page_urls(soup):
    #this function takes the search page and returns the urls of each line item
    try:
        links = soup.find_all('a', class_='s-item__link')
    except:
        links = []
    urls = [item.get('href') for item in links]
    #http://localhost:8888/notebooks/Users/lmoran/Data_Projects/Amazon_Prices/1.Web-scraping%20Amazon.ipynb#
    return urls
    

In [231]:
def write_csv(data, link):
    #this function adds the datapoints to the csv
    with open('output4.csv', 'a') as csvfile:
        writer = csv.writer(csvfile)
        
        row = [data['name'], data['price'], data['units'], 
               data['ratings_num'], data['seller_name'],
               data['seller_ratings'], data['seller_feedback_score'],
               data['condition1'], data['brand'], data['color'], data['viewed_ph']]
        
        writer.writerow(row)

In [238]:
def main():
    
    #i have run this with 4 url page and will need to combine the csvs 
    url = 'https://www.ebay.com/sch/i.html?_nkw=watches&_ipg=200&_pgn=6'
    products = get_page_urls(url_request(url))
        

    for links in products:
        data = get_detail_data(url_request(links))
        write_csv(data, links)
        #print(data)
    

In [239]:
if __name__ == '__main__':
    main()

In [243]:
filenames = ['output1.csv','output2.csv','output3.csv','output4.csv']

combined_csv = pd.concat( [ pd.read_csv(f,encoding='cp1252') for f in filenames ] )

combined_csv.to_csv('dataset.csv')